# 프로젝트 2 : 날씨 좋은 월요일 오후 세 시, 자전거 타는 사람은 몇 명?

In [ ]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [ ]:
# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import datetime as dt

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler

#evaluation metrics
from sklearn.metrics import mean_squared_log_error,mean_squared_error # for regression


## (1) 데이터 가져오기

In [ ]:
## Local로 작업

In [ ]:
train_df = pd.read_csv('./data/bike_train.csv')
test_df = pd.read_csv('./data/bike_test.csv')
df = train_df.copy()

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.head()

**Data Fields**. 
- datetime - hourly date + timestamp  
- season -  1 = spring, 2 = summer, 3 = fall, 4 = winter 
- holiday - whether the day is considered a holiday
- workingday - whether the day is neither a weekend nor holiday
- weather - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
-           2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
-           3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
-           4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog 
- temp - temperature in Celsius
- atemp - "feels like" temperature in Celsius
- humidity - relative humidity
- windspeed - wind speed
- casual - number of non-registered user rentals initiated
- registered - number of registered user rentals initiated
- count - number of total rentals

In [ ]:
df.info()

### Null Value Check

In [ ]:
df.isnull().sum()

In [ ]:
msno.matrix(df)

## (2) datetime 컬럼을 datetime 자료형으로 변환하고 연, 월, 일, 시, 분, 초까지 6가지 컬럼 생성하기

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])

In [ ]:
#weekday 도 중요 피쳐라 추가로 축출
datetime_col = ['year','month','day','weekday','hour','minute','second']

for col in datetime_col:
    df[col] = getattr(df['datetime'].dt, col)

## (3) year, month, day, hour, minute, second 데이터 개수 시각화하기

### Count_Plot

In [ ]:
nrows, ncols = len(datetime_col), 1
fig, axs = plt.subplots(nrows,ncols,figsize=(9,3*nrows))
for i in range(len(datetime_col)):
    sns.set_theme(style="whitegrid")
    sns.countplot(x=df[datetime_col[i]],data=df, ax=axs[i])
    axs[i].set_title('Count Plot of {}'.format(datetime_col[i]))
plt.tight_layout()
plt.show()

### Bar_Plot

In [ ]:
#counter plot 으로는 데이터 구분이 힘들어 bar_plot 으로 다시 그림, counter 와 다른 양상을 지닌, casual 도 같이 그림
fig, axs = plt.subplots(nrows,ncols,figsize=(9,3*nrows))
for i in range(len(datetime_col)):
    sns.barplot(data=df,x=datetime_col[i],y='count',ax=axs[i],alpha=0.5)
    sns.barplot(data=df,x=datetime_col[i],y='casual',ax=axs[i],alpha=0.5,color='red')
    axs[i].set_title('{} - BarPlot'.format(datetime_col[i]))
    axs[i].set_ylabel('count/casual')
plt.tight_layout()
plt.show()

## (4) X, y 컬럼 선택 및 train/test 데이터 분리

### Data Backfill - 결측치 채우기 

In [ ]:
df['windspeed'].value_counts()

In [ ]:
df['windspeed'] = df['windspeed'].replace(0,np.nan)
df['windspeed'].isnull().sum()

In [ ]:
df['windspeed'].fillna(method='bfill',inplace=True)
df['windspeed'].isnull().sum()
df['windspeed'].value_counts()

### Outlier : IQR - Count 만 적용

In [ ]:
sns.boxplot(data=df,y='count')

In [ ]:
sns.boxplot(data=df,y='count',x='hour')

In [ ]:
Q1 = df['count'].quantile(0.25)
Q3 = df['count'].quantile(0.75)

print(Q1,Q3)

IQR = Q3-Q1

In [ ]:
filter = (df['count'] >= Q1 - 1.5 * IQR) & (df['count'] < Q3 + 1.5 *IQR )

display(df.loc[filter].shape)
display(df.shape)

df_clean = df.loc[filter]

In [ ]:
df = df_clean.copy()

### Correlation 

In [ ]:
clean_col = [ 'weather', 'temp', 'humidity', 'windspeed', 
              'year','season', 'month', 'weekday', 'holiday', 'workingday', 'hour',
              'casual', 'registered', 'count']

correlation = df[clean_col].corr()

fig, ax = plt.subplots(figsize=(20,10))
mask = np.array(correlation)
mask[np.tril_indices_from(mask)] = False
sns.heatmap(correlation,square=True,annot=np.round(correlation,2),mask=mask,cmap='coolwarm')

In [ ]:
correlation.index.name = 'category'
correlation.T.index.name = 'relevant_category'

corr_df = pd.DataFrame(correlation.stack(),columns=['correaltion']).reset_index()
corr_df = corr_df.loc[corr_df['category'] != corr_df['relevant_category']]
corr_df['rank'] = corr_df.groupby('category')['correaltion'].rank('min',ascending=False).astype(int)
corr_df.sort_values('correaltion',ascending=False)

#### [count - features] correlation rank

In [ ]:
corr_df[corr_df['category']=='count'].sort_values('rank')

#### [casual - features] correlation rank

In [ ]:
corr_df[corr_df['category']=='casual'].sort_values('rank')

#### [registered - features] correlation rank

In [ ]:
corr_df[corr_df['category']=='registered'].sort_values('rank')

In [ ]:
# month, season ,windspeed 는 count, casual, registered 모두에게 corr point 가 낮다.

### Min-Max Scaling : 수치형 자료 

In [ ]:
score_cols = ['temp','humidity','windspeed']

def min_max_scaling(data):
    return (data - data.min()) / (data.max() - data.min())

df[score_cols] = df[score_cols].apply(min_max_scaling)

### One-Hot Encoding : 범주형 자료 

In [ ]:
category_col = ['weather', 'month', 'weekday', 'holiday', 'workingday', 'hour']

for col in category_col:
    df[col] = df[col].astype('category')

weather_df = pd.get_dummies(df['weather'],prefix='weather')
weather_df = weather_df.astype(int)
df=pd.concat([df,weather_df],axis=1)

season_df = pd.get_dummies(df['season'],prefix='season')
season_df = season_df.astype(int)
df=pd.concat([df,season_df],axis=1)

In [ ]:
# year -> 2011 : 0 , 2012 : 1
df.loc[ df['year'] == 2011 ,'year'] = 0
df.loc[ df['year'] == 2012 ,'year'] = 1

### Feature Selection 

#### 1차 Feature Selection : count, registered, casual 모두에게 영향을 미치지 않는 column 축출

In [ ]:
clean_col = [ 'holiday', 'workingday', 'temp',
              'humidity', 'windspeed', 'count',
              'year', 'weekday', 'hour','month',
              'weather_1', 'weather_2', 'weather_3', 'weather_4',
              'season_1','season_2', 'season_3', 'season_4'
            ]

df[clean_col]

## (5) LinearRegression 모델 학습 : Count Predict -> Casual + Registered Predict Model 로 전환

#### Train-Test Set 

In [ ]:
X = df[clean_col].drop(['count'],axis=1)
y = df[['casual','registered']]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=36)

### Casual Predict

#### LinearRegression 모델 학습 - Casual

In [ ]:
model = LinearRegression()
model.fit(X_train,y_train['casual'])

In [ ]:
y_pred_casual = model.predict(X_test)

In [ ]:
print("Coefficients: ", model.coef_)
print("Intercept: ", model.intercept_)

### Registered Predict

#### LinearRegression 모델 학습 - Registered

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(X_train,y_train['registered'])

In [ ]:
y_pred_registered = model.predict(X_test)

In [ ]:
print("Coefficients: ", model.coef_)
print("Intercept: ", model.intercept_)

### Count = Casual + Predict

In [ ]:
y_pred_count = y_pred_casual + y_pred_registered       #count(예측) = casual + registered
y_test_count = y_test['casual'] + y_test['registered'] #count(test) = casual + registered

## (6) 학습된 모델로 X_test에 대한 예측값 출력 및 손실함수값 계산

In [ ]:
print("MSE: ", mean_squared_error(y_test_count, y_pred_count))
print("RMSE: ",np.sqrt(mean_squared_error(y_test_count, y_pred_count)))

# 현재 RMSE : 121

1차 시도 (baseline) :
    - MSE:  20476
    - RMSE:  143

2차 시도 (min-max Scaling) : 143->122
    - MSE:  14972
    - RMSE: 122

3차 시도 (astype-Category) : 큰 변화 없음
    - MSE:  14972
    - RMSE: 122

4차 시도 (Casual + Registered) : 122->115 감소
    - MSE:  13225
    - RMSE: 115

4차 시도 (Casual + Registered) : 122->115 감소
    - MSE:  13225
    - RMSE: 115

In [ ]:
5차 시도 (One-hot Encoding) : 살짝 내려갔음 
    - MSE:  13080
    - RMSE:  114

In [ ]:
6차 시도 (feature_selection(windspeed,season,month)) : RMSE 오히려 증가, MSE 감소
    - MSE:  15260
    - RMSE:  123

In [ ]:
7차 시도 (feature 내 True/False 값 -> 1,0 변환 ) : RMSE 증가
    - MSE:  17260
    - RMSE:  130

In [ ]:
8차 시도 (Year 값 -> 1,0 변환 ) : RMSE 감소
    - MSE:  14700
    - RMSE:  121

## (7) x축은 temp 또는 humidity로, y축은 count로 예측 결과 시각화하기

In [ ]:
sns.scatterplot(x=X_test['temp'],y=y_pred_count)

In [ ]:
sns.scatterplot(x=X_test['humidity'],y=y_pred_count)